In [1]:
from pathlib import Path

import torch
import numpy as np
import random
import pickle
from absl import logging
from absl.flags import FLAGS
from cellot import losses
from cellot.utils.loaders import load
from cellot.models.cellot import compute_loss_f, compute_loss_g, compute_w2_distance
from cellot.train.summary import Logger
from cellot.data.utils import cast_loader_to_iterator
from cellot.models.ae import compute_scgen_shift
from tqdm import trange

/data/rsg/chemistry/johnyang/miniconda3/envs/cellot/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
TARGET = 'all' # 'all' denotes all drugs

In [3]:
import omegaconf

yaml_str = f"""
model:
   name: scgen
   beta: 0.0
   dropout: 0.0
   hidden_units: [512, 512]
   latent_dim: 50

optim:
   lr: 0.001
   optimizer: Adam
   weight_decay: 1.0e-05

scheduler:
   gamma: 0.5
   step_size: 100000

training:
  cache_freq: 10000
  eval_freq: 2500
  logs_freq: 250
  n_iters: 250000

data:
  type: cell
  source: control
  condition: drug
  path: /Mounts/rbg-storage1/users/johnyang/cellot/datasets/scrna-sciplex3/hvg.h5ad
  target: {TARGET}

datasplit:
    groupby: drug   
    name: train_test
    test_size: 0.2
    random_state: 0

dataloader:
    batch_size: 256
    shuffle: true
"""

config = omegaconf.OmegaConf.create(yaml_str)

### Utils

In [4]:
def load_lr_scheduler(optim, config):
    if "scheduler" not in config:
        return None

    return torch.optim.lr_scheduler.StepLR(optim, **config.scheduler)

def check_loss(*args):
    for arg in args:
        if torch.isnan(arg):
            raise ValueError


def load_item_from_save(path, key, default):
    path = Path(path)
    if not path.exists():
        return default

    ckpt = torch.load(path)
    if key not in ckpt:
        logging.warn(f"'{key}' not found in ckpt: {str(path)}")
        return default

    return ckpt[key]

In [5]:
import cellot.models
from cellot.data.cell import load_cell_data


def load_data(config, **kwargs):
    data_type = config.get("data.type", "cell")
    if data_type in ["cell", "cell-merged", "tupro-cohort"]:
        loadfxn = load_cell_data

    elif data_type == "toy":
        loadfxn = load_toy_data

    else:
        raise ValueError

    return loadfxn(config, **kwargs)


def load_model(config, restore=None, **kwargs):
    name = config.model.name
    if name == "cellot":
        loadfxn = cellot.models.load_cellot_model

    elif name == "scgen":
        loadfxn = cellot.models.load_autoencoder_model

    elif name == "cae":
        loadfxn = cellot.models.load_autoencoder_model

    elif name == "popalign":
        loadfxn = cellot.models.load_popalign_model

    else:
        raise ValueError

    return loadfxn(config, restore=restore, **kwargs)


def load(config, restore=None, include_model_kwargs=False, **kwargs):

    loader, model_kwargs = load_data(config, include_model_kwargs=True, **kwargs)

    model, opt = load_model(config, restore=restore, **model_kwargs)

    if include_model_kwargs:
        return model, opt, loader, model_kwargs

    return model, opt, loader

### Training

In [6]:
def train_auto_encoder(outdir, config):
    def state_dict(model, optim, **kwargs):
        state = {
            "model_state": model.state_dict(),
            "optim_state": optim.state_dict(),
        }

        if hasattr(model, "code_means"):
            state["code_means"] = model.code_means

        state.update(kwargs)

        return state

    def evaluate(vinputs):
        with torch.no_grad():
            loss, comps, _ = model(vinputs)
            loss = loss.mean()
            comps = {k: v.mean().item() for k, v in comps._asdict().items()}
            check_loss(loss)
            logger.log("eval", loss=loss.item(), step=step, **comps)
        return loss

    logger = Logger(outdir / "cache/scalars")
    cachedir = outdir / "cache"
    model, optim, loader = load(config, restore=cachedir / "last.pt")

    iterator = cast_loader_to_iterator(loader, cycle_all=True)
    scheduler = load_lr_scheduler(optim, config)

    n_iters = config.training.n_iters
    step = load_item_from_save(cachedir / "last.pt", "step", 0)
    if scheduler is not None and step > 0:
        scheduler.last_epoch = step

    best_eval_loss = load_item_from_save(
        cachedir / "model.pt", "best_eval_loss", np.inf
    )

    eval_loss = best_eval_loss

    ticker = trange(step, n_iters, initial=step, total=n_iters)
    for step in ticker:

        model.train()
        inputs = next(iterator.train)
        optim.zero_grad()
        loss, comps, _ = model(inputs)
        loss = loss.mean()
        comps = {k: v.mean().item() for k, v in comps._asdict().items()}
        loss.backward()
        optim.step()
        check_loss(loss)

        if step % config.training.logs_freq == 0:
            # log to logger object
            logger.log("train", loss=loss.item(), step=step, **comps)

        if step % config.training.eval_freq == 0:
            model.eval()
            eval_loss = evaluate(next(iterator.test))
            if eval_loss < best_eval_loss:
                best_eval_loss = eval_loss
                sd = state_dict(model, optim, step=(step + 1), eval_loss=eval_loss)

                torch.save(sd, cachedir / "model.pt")

        if step % config.training.cache_freq == 0:
            torch.save(state_dict(model, optim, step=(step + 1)), cachedir / "last.pt")

            logger.flush()

        if scheduler is not None:
            scheduler.step()

    if config.model.name == "scgen" and config.get("compute_scgen_shift", True):
        labels = loader.train.dataset.adata.obs[config.data.condition]
        compute_scgen_shift(model, loader.train.dataset, labels=labels)

    torch.save(state_dict(model, optim, step=step), cachedir / "last.pt")

    logger.flush()

### Outdir

In [10]:
import torch
import GPUtil
import os

def get_free_gpu():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(device)
    # Set environment variables for which GPUs to use.
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    chosen_gpu = ''.join(
        [str(x) for x in GPUtil.getAvailable(order='memory')])
    os.environ["CUDA_VISIBLE_DEVICES"] = chosen_gpu
    print(f"Using GPUs: {chosen_gpu}")
    return chosen_gpu

In [11]:
device = f'cuda:{get_free_gpu()}'

cuda
Using GPUs: 1


In [12]:
our_ae_sd = torch.load('/Mounts/rbg-storage1/users/johnyang/cellot/results/sciplex3/full_ae/cache/model.pt', map_location=device)

In [13]:
from pathlib import Path
outdir_path = '/Mounts/rbg-storage1/users/johnyang/cellot/results/sciplex3/full_ae'
outdir = Path(outdir_path)

In [15]:
from tqdm import tqdm

In [16]:
def evaluate(vinputs, tmodel):
    with torch.no_grad():
        loss, comps, _ = tmodel(vinputs)
        loss = loss.mean()
        comps = {k: v.mean().item() for k, v in comps._asdict().items()}
        check_loss(loss)
        logger.log("eval", loss=loss.item(), step=step, **comps)
    return loss

logger = Logger(outdir / "cache/scalars")
cachedir = outdir / "cache"
model, optim, loader = load(config, restore=cachedir / "last.pt")

iterator = cast_loader_to_iterator(loader, cycle_all=True)
scheduler = load_lr_scheduler(optim, config)

n_iters = config.training.n_iters
step = load_item_from_save(cachedir / "last.pt", "step", 0)
if scheduler is not None and step > 0:
    scheduler.last_epoch = step

best_eval_loss = load_item_from_save(
    cachedir / "model.pt", "best_eval_loss", np.inf
)

eval_loss = best_eval_loss

# ticker = trange(step, n_iters, initial=step, total=n_iters)
eval_losses = []

for batch in tqdm(iterator.test):
    model.eval()
    eval_loss = evaluate(batch, model)
    eval_losses.append(eval_loss)
    if eval_loss < best_eval_loss:
        best_eval_loss = eval_loss
        
    print(eval_loss)
        # sd = state_dict(model, optim, step=(step + 1), eval_loss=eval_loss)
        # torch.save(sd, cachedir / "model.pt")

2023-06-23 11:11:34,456 Loaded cell data with TARGET all and OBS SHAPE (762039, 16)
/tmp/ipykernel_2939940/847900366.py:20: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn(f"'{key}' not found in ckpt: {str(path)}")
2023-06-23 11:11:41,242 'best_eval_loss' not found in ckpt: /Mounts/rbg-storage1/users/johnyang/cellot/results/sciplex3/full_ae/cache/model.pt
17it [00:00, 84.19it/s]

tensor(0.0860)
tensor(0.0804)
tensor(0.0832)
tensor(0.0882)
tensor(0.0783)
tensor(0.0766)
tensor(0.0800)
tensor(0.0953)
tensor(0.0757)
tensor(0.0959)
tensor(0.0762)
tensor(0.0791)
tensor(0.0719)
tensor(0.0879)
tensor(0.0803)
tensor(0.0721)
tensor(0.0778)
tensor(0.0788)
tensor(0.0848)
tensor(0.0878)


35it [00:00, 85.72it/s]

tensor(0.0724)
tensor(0.0868)
tensor(0.0891)
tensor(0.0806)
tensor(0.0797)
tensor(0.0769)
tensor(0.0789)
tensor(0.0867)
tensor(0.0768)
tensor(0.0826)
tensor(0.0850)
tensor(0.0835)
tensor(0.0837)
tensor(0.0712)
tensor(0.0701)
tensor(0.0794)
tensor(0.0864)
tensor(0.0797)


54it [00:00, 88.34it/s]

...

1722it [00:19, 89.85it/s]

tensor(0.0744)
tensor(0.0847)
tensor(0.0745)
tensor(0.0783)
tensor(0.0829)
tensor(0.0778)
tensor(0.0794)
tensor(0.0796)
tensor(0.0845)
tensor(0.0769)
tensor(0.0858)
tensor(0.0718)
tensor(0.0780)
tensor(0.0805)
tensor(0.0852)
tensor(0.0839)
tensor(0.0766)
tensor(0.0757)
tensor(0.0756)


1744it [00:19, 89.56it/s]

tensor(0.0841)
tensor(0.0771)
tensor(0.0876)
tensor(0.0757)
tensor(0.0806)
tensor(0.0750)
tensor(0.0849)
tensor(0.0792)
tensor(0.0825)
tensor(0.0794)
tensor(0.0800)
tensor(0.0854)
tensor(0.0866)
tensor(0.0766)
tensor(0.0752)


KeyboardInterrupt: 